In [12]:


import pandas as pd
import os

from nltk.app.wordnet_app import explanation

output_dir = '../performance'


import pandas as pd
import os
import json

output_dir = '../performance'

results = []
for file in os.listdir(output_dir):
    with open(f'{output_dir}/{file}', 'r') as f:
        response = f.read()
        
        # remove all that is not json
        response = response.split('{')[-1].split('}')[0]
        response = '{' + response + '}'
        
        response = response.replace('\'', '\"')
        
        try:
            response = json.loads(response)
        except json.JSONDecodeError:
            continue

        score = response.get('score', None)
        relevant = response.get('relevant', None)
        explanation = response.get('explanation', None)
        keywords = response.get('keywords', None)
        
       
        index, query = file.split('_')
        query = query.split('.')[0]
  

        results.append({
            'index': index,
            'query': query,
            'score': score,
            'relevant': relevant,
            'explanation': explanation,
            'keywords': keywords
        })

df_results = pd.DataFrame(results)
df_results.to_csv('results.csv', index=False)
